In [ ]:
import json
import os
import glob

In [ ]:
try:
    os.mkdir('out')
except FileExistsError:
    pass

file = open('data', 'r')
frames = {}
lines = file.readlines()
for idx, line in enumerate(lines):
    try:
        data = json.loads(line)
        _ = data['frame']
    except:
        continue
    frame = data['frame']
    
    try:
        _ = frames[data['ssi']]
    except:
        frames[data['ssi']] = {}
        
    usage_marker = str(data['downlink usage marker'])
    try:
        _ = frames[data['ssi']][usage_marker]
    except:
        frames[data['ssi']][usage_marker] = []
    
    try:
        last_frame_idx = int(frames[data['ssi']][usage_marker][-1].split('.')[0].split('/')[-1])
        if lines[last_frame_idx] == frame:
            continue
    except:
        pass
    
    frames[data['ssi']][usage_marker].append(f'out/{str(idx).zfill(5)}.ogg')
    
    command = f"echo -n \"{frame}\" | base64 -d | perl -MCompress::Zlib -e 'undef $/; print uncompress(<>)' | cdecoder /dev/stdin /dev/stdout | sdecoder /dev/stdin out/{str(idx).zfill(5)}.raw"
    os.system(command)

In [ ]:
for file in sorted(glob.glob('out/*.raw')):
    out = file.split('.')[0] + ".ogg"
    os.system(f"sox -q -r 8000 -e signed -b 16 -c 1 {file} {out}")
    print(out)

In [ ]:
try:
    os.mkdir('decoded')
except FileExistsError:
    pass

for ssi in frames:
    for marker in frames[ssi]:  
        os.system('sox ' + ' '.join(frames[ssi][marker]) + f' decoded/{ssi}.{marker}.ogg')